-------------- Real Project Time -------------

Use tensorflow.keras library for models. And you are free to use chatgpt here at this stage - Yayyyyyyyy\
Use it where you need help, not to complete the assignment lol

In [10]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
import os
from sklearn.model_selection import train_test_split

Define a function to extract features from the given filename. You will have to use librosa library for this. You can vary the features however you like. The preferred features are mfcc, chroma and melspectogram.

In [6]:
def extract_features(file_path):
    # load file
    audio, sr = librosa.load(file_path)
    
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfccs = np.mean(mfccs.T, axis=0)
    
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft, sr=sr)
    chroma = np.mean(chroma.T, axis=0)
    
    melspec = librosa.feature.melspectrogram(y=audio, sr=sr)
    melspec = np.mean(melspec.T, axis=0)
    
    
    return mfccs, chroma, melspec

Now, load the data from the database.

You can find the database from https://zenodo.org/records/1188976#.XsAXemgzaUk

You need to only download the audio files and not the video files. Also, read up on the naming convention of the files from the webpage.

In [14]:
def one_hot(y):
    temp = np.zeros(8)
    temp[y] = 1
    return temp

In [15]:
def load_data(test_size):
    features = []
    labels = []
    
    for i in range(1,25):
        I = "{:02}".format(i)
        audio_dir_path = f'./Audio_Speech/Actor_{I}'
        
        for file_name in (os.listdir(path=f'./Audio_Speech/Actor_{I}')):
            file_path = os.path.join(audio_dir_path, file_name)
            mfccs, chroma, mel = extract_features(file_path)
            
            feature_row = np.hstack([mfccs, chroma, mel])
            features.append(feature_row)
            
            label = one_hot(int(file_name.split('-')[3]))
            labels.append(label)
            
    features = np.array(features)
    labels = np.array(labels)
    
    #splitting
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=12)
    return X_train, X_test, y_train, y_test

Use the load_data function to load data and split the data using the train_test_split function.

In [16]:
#split the dataset into training and testing data with testing data = 0.2 of total data
X_train, X_test, y_train, y_test = load_data(test_size=0.2)

Now you need to define the model for this computation.
For this, we will use simple neural network for this.\
The model will have a hidden layer of 300 nodes and an output layer of [number of emotions] nodes.\
You have to use relu as activation function for the hidden layer and softmax for the output layer.\
Keep the loss categorical-crossentropy, optimizer as adam and accuracy as metric.\
You can use batch-size as 256 and epochs as 300, but it is left upto you :)

In [ ]:
#define the model.
model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Now everything is easy-peezy. Just fit the model on training data as you usually do, predict for the testing data, print the accuracy. Hehe

In [20]:
#fit the model on training data with validation split of 0.2
history = model.fit(
    X_train,
    y_train,
    epochs=300,
    batch_size=256,
    validation_split=0.2
)

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5110 - loss: 4.3107 - val_accuracy: 0.5801 - val_loss: 2.8278
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5472 - loss: 2.3569 - val_accuracy: 0.5931 - val_loss: 0.7822
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6694 - loss: 0.9763 - val_accuracy: 0.7056 - val_loss: 0.6547
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6271 - loss: 0.9448 - val_accuracy: 0.7619 - val_loss: 0.6933
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7165 - loss: 0.8746 - val_accuracy: 0.5758 - val_loss: 0.8910
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6220 - loss: 0.8297 - val_accuracy: 0.7706 - val_loss: 0.5531
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6785 - loss: 0.6934 - val_accuracy: 0.7619 - val_loss: 0.6243
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7111 - loss: 0.7019 - val_accuracy: 0.5628 - val_loss:

In [21]:
#predict for testing data and printout the accuracy
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy of the model on X_test:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7593 - loss: 0.8032 
Accuracy of the model on X_test: 0.7673611044883728


Now comes the learning part :)\
Here's a new thing. Print the classification report of your predictions.

In [32]:
#print the classification report for the model
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

class_names = ['Neutral','Calm','Happy','Sad','Angry','Fearful','Disgust','Surprised']
print(type(class_names))
report = classification_report(y_test_classes, y_pred_classes, target_names=class_names, labels=range(len(class_names)), zero_division=1)
print(report)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
<class 'list'>
              precision    recall  f1-score   support

     Neutral       1.00      1.00      1.00         0
        Calm       0.71      0.94      0.81       149
       Happy       0.90      0.58      0.71       139
         Sad       1.00      1.00      1.00         0
       Angry       1.00      1.00      1.00         0
     Fearful       1.00      1.00      1.00         0
     Disgust       1.00      1.00      1.00         0
   Surprised       1.00      1.00      1.00         0

    accuracy                           0.77       288
   macro avg       0.95      0.94      0.94       288
weighted avg       0.80      0.77      0.76       288



Just for fun. You can explore on callbacks used in model.fit().\
For resources, you can go to https://keras.io/api/callbacks/ and explore on your own. Or just chatgpt :)
Now try the model using EarlyStopping callback, LearningRateScheduler callback and ModelCheckpoint callback.

In [ ]:
#Have fun ..... you just completed the project

The timeline for this is 2 weeks. In the last two weeks, we will be exploring further concepts in AIML or doing some fun activities in the field. Look forward to it :)